In [ ]:
import cv2
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from SIFT_FLANNMatcher import sift_num_record, sift_acc_record

In [ ]:
imgnames = []
imgnames = os.listdir("D:/Python3/FeatureMatch/RectifiedImages2")
# savepath = 'D:/Python3/FeatureMatch/SURFMatchedImages2/'
start = time.clock()
halfnum = int(0.5*len(imgnames))
surf_num_record = []
surf_acc_record = []
for i in range(30):
    imgname1 = 'D:/Python3/FeatureMatch/RectifiedImages2/'+imgnames[i]
    imgname2 = 'D:/Python3/FeatureMatch/RectifiedImages2/'+imgnames[i + halfnum]
    surf = cv2.xfeatures2d.SURF_create()

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    img1 = cv2.imread(imgname1)
    kp1, des1 = surf.detectAndCompute(img1, None)

    img2 = cv2.imread(imgname2)
    kp2, des2 = surf.detectAndCompute(img2, None)
    matches = flann.knnMatch(des1, des2, k=2)
    goodones = []
    truthground = []

    for m, n in matches:
        if m.distance < 0.6 * n.distance:
            truthground.append([m])
        elif m.distance < 0.7 * n.distance:
            goodones.append([m])

    num = len(goodones) + len(truthground)
    surf_num_record.append(num)
    acc = len(truthground) / num
    surf_acc_record.append(acc)

    print('Matched num of %s and %s is %d,acc = %.3f' %(imgnames[i], imgnames[i + halfnum], num, acc))
    img = cv2.drawMatchesKnn(img1, kp1, img2, kp2, goodones, None, flags=2)
    cv2.imshow("SURF_FLANN", img)
    # cv2.imwrite(savepath + imgnames[i][-8:-4] + '-' + str(len(goodones)) + '.bmp', img)

In [ ]:
# 绘图
order = np.arange(1, 31, 1)
plt.subplot(2, 1, 1)
plt.plot(order, sift_num_record, label='sift_matched_num')
plt.plot(order, surf_num_record, label='surf_matched_num')
plt.title('SIFT and SURF matched num')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(order, sift_acc_record, label='sift_matched_acc')
plt.plot(order, surf_acc_record, label='surf_matched_acc')
plt.title('SIFT and SURF matched acc')
plt.legend()
plt.show()

end = time.clock()
print('Running time of All: %s Seconds' % (end-start))